In [3]:
import torch
import torch.nn as nn
import numpy as np

# 01. Pytorch实现注意力机制、多头注意力与自注意力

## 注意力机制

In [125]:
# 01. Pytorch实现注意力机制、多头注意力与自注意力
class ScaledDotProductAttention(nn.Module):
    """ Scaled Dot-Product Attention """

    def __init__(self, scale):
        super().__init__()

        self.scale = scale
        self.softmax = nn.Softmax(dim=2)  # 仅对最后一个维度进行softmax，这确保了每个查询（q）位置对所有键（k）的注意力权重之和为1

    def forward(self, q, k, v, mask=None):
        print(q.shape, k.shape, k.transpose(1, 2).shape)
        u = torch.bmm(q, k.transpose(1, 2)) # 1.Matmul 矩阵乘法：计算查询矩阵(q)和键矩阵(k)的点积，使用torch.bmm进行批量矩阵乘法
        print('u.shape', u.shape)
        u = u / self.scale # 2.Scale 缩放：将点积结果除以缩放因子（通常是键向量维度的平方根， 因为假设了方差为1， 均值为0， 所以除以根号d_k）

        if mask is not None:
            u = u.masked_fill(mask, -np.inf) # 3.Mask 掩码，将false的位置填充为-inf， 这样在softmax时，这些位置的值会趋近于0，实现了注意力的选择性关注；
        print('u.shape after mask', u.shape)

        attn = self.softmax(u) # 4.Softmax 归一化，将u中的值归一化到0-1之间， 这样每个位置的值表示该位置与其他位置的关联度；
        print('attn.shape', attn.shape)
        output = torch.bmm(attn, v) # 5.Output 加权求和，将归一化后的注意力权重与值矩阵相乘，得到加权求和的结果；
        print('output.shape', output.shape)

        return attn, output

if __name__ == "__main__":
    n_q, n_k, n_v = 2, 4, 4
    d_q, d_k, d_v = 128, 128, 64
    batch = 1
    q = torch.randn(batch, n_q, d_q)
    k = torch.randn(batch, n_k, d_k)
    v = torch.randn(batch, n_v, d_v)
    mask = torch.zeros(batch, n_q, n_k).bool()

    attention = ScaledDotProductAttention(scale=np.power(d_k, 0.5))
    attn, output = attention(q, k, v, mask=mask)

    print(attn.shape)
    print(output.shape)

torch.Size([1, 2, 128]) torch.Size([1, 4, 128]) torch.Size([1, 128, 4])
u.shape torch.Size([1, 2, 4])
u.shape after mask torch.Size([1, 2, 4])
attn.shape torch.Size([1, 2, 4])
output.shape torch.Size([1, 2, 64])
torch.Size([1, 2, 4])
torch.Size([1, 2, 64])


## 多头注意力机制

In [135]:
import math
import torch.nn.functional as F
class MultiHeadSelfAttention(nn.Module):
    """多头自注意力机制
    特点：
        1. 每个头有自己独特的参数（投影矩阵的一部分）
        2. 每个头关注输入的不同方面
        3. 最后的输出投影层将所有头的输出组合在一起
    
    参数:
        dim_in (int): 输入维度
        dim_k (int): 键和查询的维度
        dim_v (int): 值的维度
        num_heads (int): 注意力头的数量
    
    注意:
        - dim_k 和 dim_v 必须能被 num_heads 整除
        - 每个头的维度为: dk = dim_k // num_heads, dv = dim_v // num_heads
    """
    
    def __init__(self, dim_in, dim_k, dim_v, num_heads=8):
        super().__init__()
        
        # 参数检查
        if dim_k % num_heads != 0 or dim_v % num_heads != 0:
            raise ValueError("dim_k和dim_v必须是num_heads的整数倍")
            
        # 保存配置参数
        self.dim_in = dim_in
        self.dim_k = dim_k 
        self.dim_v = dim_v
        self.num_heads = num_heads
        self.head_dim_k = dim_k // num_heads  # 每个头的键/查询维度
        self.head_dim_v = dim_v // num_heads  # 每个头的值维度
        
        # 线性变换层
        self.q_proj = nn.Linear(dim_in, dim_k, bias=False)
        self.k_proj = nn.Linear(dim_in, dim_k, bias=False)
        self.v_proj = nn.Linear(dim_in, dim_v, bias=False)
        self.output_proj = nn.Linear(dim_v, dim_in)  # 新增：输出投影
        
        # 缩放因子
        self.scale = 1 / math.sqrt(self.head_dim_k)  # 从math模块导入sqrt
        
        # 初始化参数
        self._reset_parameters()
    
    def _reset_parameters(self):
        """初始化模型参数"""
        # 使用xavier初始化提高训练稳定性
        """Xavier初始化（也叫Glorot初始化）的目的是：
            保持每一层输入和输出的方差大致相等
            防止深层网络中的梯度消失或爆炸
            计算方法：权重从均值为0，方差为2/(fan_in + fan_out)的均匀分布中采样
            fan_in：输入特征数
            fan_out：输出特征数"""
        nn.init.xavier_uniform_(self.q_proj.weight)
        nn.init.xavier_uniform_(self.k_proj.weight)
        nn.init.xavier_uniform_(self.v_proj.weight)
        nn.init.xavier_uniform_(self.output_proj.weight)
    
    def _reshape_to_batches(self, x, batch_size, seq_len, dim):
        """将输入重塑为多头格式"""
        # x: (batch, seq_len, dim)
        x = x.reshape(batch_size, seq_len, self.num_heads, dim // self.num_heads)
        # 原始: (batch, seq_len, 512)
        # reshape后: (batch, seq_len, 8, 64)
        # 每个头的梯度会传回到对应的投影矩阵部分
        # 不同头的参数会得到不同的梯度更新

        # 转置以让head维度在前
        x = x.transpose(1, 2)  # (batch, num_heads, seq_len, head_dim)
        return x
    
    def forward(self, x, mask=None):
        """前向传播
        
        参数:
            x: 输入张量, shape (batch_size, seq_len, dim_in)
            mask: 可选的注意力掩码, shape (batch_size, seq_len, seq_len)
        
        返回:
            output: 输出张量, shape (batch_size, seq_len, dim_in)
            attention: 注意力权重, shape (batch_size, num_heads, seq_len, seq_len)
        """
        batch_size, seq_len, dim_in = x.shape
        assert dim_in == self.dim_in, f"输入维度{dim_in}与期望维度{self.dim_in}不匹配"
        
        # 1. 线性投影
        q = self.q_proj(x)  # (batch, seq_len, dim_k)
        k = self.k_proj(x)  # (batch, seq_len, dim_k)
        v = self.v_proj(x)  # (batch, seq_len, dim_v)
        
        # 2. 重塑为多头形式
        q = self._reshape_to_batches(q, batch_size, seq_len, self.dim_k)  # (batch, num_heads, seq_len, seq_len)
        k = self._reshape_to_batches(k, batch_size, seq_len, self.dim_k)  # (batch, num_heads, seq_len, seq_len)
        v = self._reshape_to_batches(v, batch_size, seq_len, self.dim_v)  # (batch, num_heads, seq_len, seq_len)
        
        # 3. 计算注意力分数
        attention_scores = torch.matmul(q, k.transpose(-2, -1))  # (batch, num_heads, seq_len, seq_len)
        attention_scores = attention_scores * self.scale
        
        # 4. 应用掩码（如果提供）
        if mask is not None:
            # 扩展mask以适应多头
            mask = mask.unsqueeze(1)  # (batch, 1, seq_len, seq_len)
            attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))
        
        # 5. 注意力权重
        attention_weights = F.softmax(attention_scores, dim=-1)
        
        # 6. 计算输出
        output = torch.matmul(attention_weights, v)  # (batch, num_heads, seq_len, head_dim_v)
        
        # 7. 重组多头输出
        output = output.transpose(1, 2)  # (batch, seq_len, num_heads, head_dim_v)
        output = output.reshape(batch_size, seq_len, self.dim_v)  # (batch, seq_len, dim_v)
        
        # 8. 最终输出投影
        output = self.output_proj(output)  # (batch, seq_len, dim_in)
        
        return output, attention_weights


In [138]:
# 使用示例
batch_size = 32
seq_len = 50
dim_in = 512
dim_k = 512
dim_v = 512
num_heads = 8

# 创建模型
mha = MultiHeadSelfAttention(
    dim_in=dim_in,
    dim_k=dim_k,
    dim_v=dim_v,
    num_heads=num_heads
)

# 创建输入
x = torch.randn(batch_size, seq_len, dim_in)
mask = torch.ones(batch_size, seq_len, seq_len)  # 可选的掩码

# 前向传播
output, attention = mha(x, mask)
print('output', output.shape)
print('attention', attention.shape)


output torch.Size([32, 50, 512])
attention torch.Size([32, 8, 50, 50])


# 张量创建


1. **数据类型**：
- torch.tensor会自动推断类型
- torch.Tensor默认是float32
- 明确指定类型可以避免意外

2. **内存共享**：
- from_numpy和as_tensor尽可能共享内存
- tensor总是创建新的内存

3. **设备放置**：
- 创建时指定device更高效
- 后续移动到GPU会产生复制

4. **梯度计算**：
- 只有浮点类型张量支持梯度计算
- requires_grad=True只对浮点张量有效

这些方法各有特点，选择合适的创建方式可以提高代码的效率和可维护性。



## 1. torch.tensor vs torch.Tensor


In [79]:
### torch.tensor()

x = torch.tensor([1, 2, 3])  # 推荐使用这种方式

# 特点：
# - 会根据输入数据推断数据类型
# - 创建新的张量，不共享内存
# - 更安全，更现代的API
# - 可以指定设备和数据类型

print(x.dtype, x.device, x.requires_grad)
x = torch.tensor([1, 2, 3], 
                dtype=torch.float32,
                # device='cuda:0',
                requires_grad=True)

print(x.dtype, x.device, x.requires_grad)
### torch.Tensor

x = torch.Tensor([1, 2, 3])  # 不推荐

# 特点：
# - 默认数据类型是float32
# - 是torch.FloatTensor的别名
# - 旧式API，不够灵活
print(x.dtype, x.device, x.requires_grad)


torch.int64 cpu False
torch.float32 cpu True
torch.float32 cpu False



## 2. 特定数值的张量创建


In [126]:

### 全0张量

# 方法1：zeros
x = torch.zeros(2, 3)
# 方法2：zeros_like
y = torch.zeros_like(x)  # 创建与x形状相同的全0张量
print(x.dtype, x.device, x.requires_grad)
print(y.dtype, y.device, y.requires_grad)
### 全1张量

# 方法1：ones
x = torch.ones(2, 3)
# 方法2：ones_like
y = torch.ones_like(x)
print(x.dtype, x.device, x.requires_grad)
print(y.dtype, y.device, y.requires_grad)

# empty: 不初始化
x = torch.empty(2, 3)  # 未初始化的张量
print(x)

### 指定值填充

x = torch.full((2, 3), 7.)  # 创建形状为(2,3)的张量，填充值为7
y = torch.full_like(x, 3)  # 创建与x形状相同的张量，填充值为3
print(x.dtype, x.device, x.requires_grad)
print(y.dtype, y.device, y.requires_grad)

x = torch.full((2, 3), 7)  # 创建形状为(2,3)的张量，填充值为7
y = torch.full_like(x, 3)  # 创建与x形状相同的张量，填充值为3
print(x.dtype, x.device, x.requires_grad)
print(y.dtype, y.device, y.requires_grad)


torch.float32 cpu False
torch.float32 cpu False
torch.float32 cpu False
torch.float32 cpu False
tensor([[0., 0., 0.],
        [0., 0., 0.]])
torch.float32 cpu False
torch.float32 cpu False
torch.int64 cpu False
torch.int64 cpu False



## 3. 随机张量，符合均匀分布、正态分布，伯努利，指数分布


In [102]:
### 均匀分布 光秃秃是均匀 uniform
# [0,1)均匀分布
x = torch.rand(2, 3)

# [a,b)均匀分布
x = torch.empty(2, 3)  # 创建一个2x3的张量，但不初始化值
x.uniform_(0, 1)  # 用均匀分布初始化x
print(x)


### 正态分布 n是normal

# 标准正态分布 N(0,1)
x = torch.randn(2, 3)

# 指定均值和标准差的正态分布
mean = 0.0
std = 1.0
x = torch.normal(mean, std, size=(2, 3))
x = torch.distributions.Normal(mean, std).sample((2, 3))

# 生成0-1二值分布
prob = 0.5
x = torch.bernoulli(torch.full((2, 3), prob))
print("二值分布", x)
# Dropout掩码生成
p = 0.5
mask = torch.bernoulli(torch.full((2, 3), 1 - p)) / (1 - p)
print("Dropout掩码", mask)
# 指数分布
rate = 1.0
x = torch.distributions.Exponential(rate).sample((2, 3))
print("指数分布", x)
# 泊松分布
rate = 4.0
x = torch.poisson(torch.full((2, 3), rate))
print("泊松分布", x)
# 设置随机种子以保证结果可复现
torch.manual_seed(42)

tensor([[0.2566, 0.7936, 0.9408],
        [0.1332, 0.9346, 0.5936]])
二值分布 tensor([[0., 1., 1.],
        [1., 1., 0.]])
Dropout掩码 tensor([[0., 0., 0.],
        [2., 0., 2.]])
指数分布 tensor([[0.2902, 0.2658, 0.2558],
        [0.3821, 0.9530, 0.9107]])
泊松分布 tensor([[2., 4., 5.],
        [4., 4., 0.]])


## 4. 序列张量创建

In [105]:
### 等差序列

# 方法1：arange
x = torch.arange(start=0, end=10, step=2)  # [0,2,4,6,8]
print("等差序列", x)
# 方法2：linspace
x = torch.linspace(start=0, end=10, steps=5)  # 均匀分成5份
print("等差序列", x)

### 等比序列

x = torch.logspace(start=0, end=2, steps=5)  # 10^0到10^2均匀分成5份
print("等比序列", x)


等差序列 tensor([0, 2, 4, 6, 8])
等差序列 tensor([ 0.0000,  2.5000,  5.0000,  7.5000, 10.0000])
等比序列 tensor([  1.0000,   3.1623,  10.0000,  31.6228, 100.0000])



## 5. 从其他格式转换


In [106]:
### 从NumPy数组

import numpy as np
arr = np.array([1, 2, 3])
x = torch.from_numpy(arr)  # 共享内存
y = torch.tensor(arr)      # 复制数据


### 从列表

x = torch.tensor([1, 2, 3])
y = torch.as_tensor([1, 2, 3])  # 尽可能避免复制




## 6. 特殊矩阵


In [109]:

### 单位矩阵

x = torch.eye(3)  # 3x3单位矩阵


### 对角矩阵

x = torch.diag(torch.tensor([1, 2, 3]))
print("对角矩阵\n", x)


对角矩阵
 tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])



## 7. 高级创建方法


In [128]:

### 稀疏张量

i = torch.tensor([[0, 1, 1],
                 [2, 0, 2]])
v = torch.tensor([3, 4, 5])
x = torch.sparse_coo_tensor(i, v, (3, 3))


### 复数张量

x = torch.complex(torch.rand(2, 2), torch.rand(2, 2))
x


tensor([[0.8009+0.1713j, 0.3037+0.8155j],
        [0.1234+0.3876j, 0.1526+0.2347j]])


## 8. 实际应用建议


In [134]:

# 1. **日常使用**：

# 推荐使用torch.tensor
x = torch.tensor([1, 2, 3])


# 2. **从NumPy转换**：

# 如果不需要共享内存
numpy_array = np.array([1, 2, 3])
x = torch.tensor(numpy_array)
# 如果需要共享内存
x = torch.from_numpy(numpy_array)


# 3. **性能优化**：

# 避免频繁创建新张量
new_data = torch.randn(1000, 1000)
# x = torch.zeros(1000, 1000)  # 预分配
x = torch.empty(1000, 1000)  # 预分配
x.copy_(new_data)  # 原地更新


# 4. **GPU使用**：

# 直接在GPU上创建
x = torch.tensor([1, 2, 3], device='cpu')
# 或者后续移动到GPU
# x = x.cuda()  # 或 x = x.to('cuda')


# 5. **梯度计算**：

x = torch.tensor([1., 2., 3.], requires_grad=True)



# 填充（padding）和掩码（masking）相关操作：

1. **内存效率**：
- pack_padded_sequence可以减少计算量
- masked_fill比循环填充更高效

2. **计算速度**：
- 批量操作通常比循环处理更快
- 适当的填充可以提高并行计算效率

3. **建议**：
- 对于RNN，优先使用pack_padded_sequence
- 图像处理建议使用F.pad
- 注意力机制中使用masked_fill


## 1. 序列填充操作


In [ ]:
# 这里还不是很懂。
### 1.1 pad_sequence

# 处理变长序列
sequences = [torch.randn(3, 4), torch.randn(5, 4), torch.randn(2, 4)]
padded = torch.nn.utils.rnn.pad_sequence(sequences, 
                                       batch_first=True,
                                       padding_value=0.0)

# 特点：
# - 自动将短序列填充到最长序列长度
# - 可以选择batch_first参数调整输出格式
# - 默认用0填充，可通过padding_value指定填充值
# - 适用于RNN/LSTM的批处理

### 1.2 pack_padded_sequence

# 压缩填充序列
packed = torch.nn.utils.rnn.pack_padded_sequence(padded_sequence,
                                                lengths,
                                                batch_first=True,
                                                enforce_sorted=False)

# 特点：
# - 将填充后的序列压缩，提高计算效率
# - 需要提供原始序列长度信息
# - 通常与RNN层配合使用
# - enforce_sorted控制是否需要按长度排序

### 1.3 pad_packed_sequence

# 解压压缩序列
unpacked, lengths = torch.nn.utils.rnn.pad_packed_sequence(packed_sequence,
                                                         batch_first=True,
                                                         padding_value=0.0)

# 特点：
# - 将压缩序列还原为填充形式
# - 返回填充后的序列和长度信息
# - 常用于RNN输出处理



## 2. 张量填充操作


In [ ]:

### 2.1 F.pad

import torch.nn.functional as F

# 张量边缘填充
input = torch.randn(3, 4)
padded = F.pad(input, pad=(1, 2, 3, 4), mode='constant', value=0)  # 左右上下
print(padded)
# 特点：
# - 支持多种填充模式：
#   - constant: 常数填充
#   - reflect: 反射填充
#   - replicate: 复制边缘值
#   - circular: 循环填充
# - 常用于图像处理和卷积层

### 2.2 常用填充模式示例

x = torch.randn(3, 4) 
# 常数填充
padded_constant = F.pad(x, (1, 1), mode='constant', value=0)

# 反射填充
padded_reflect = F.pad(x, (1, 1), mode='reflect')

# 复制填充
padded_replicate = F.pad(x, (1, 1), mode='replicate')

# 循环填充
padded_circular = F.pad(x, (1, 1), mode='circular')



tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0499,  0.5263, -0.0085,  0.7291,  0.0000,  0.0000],
        [ 0.0000,  0.1331,  0.8640, -1.0157, -0.8887,  0.0000,  0.0000],
        [ 0.0000,  0.1498, -0.2089, -0.3870,  0.9912,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])



## 3. 掩码操作


In [ ]:

### 3.1 masked_fill

# 条件填充
mask = torch.zeros(3, 4).bool()
tensor = torch.randn(3, 4)
masked = tensor.masked_fill(mask, value=-float('inf'))

# 特点：
# - 根据掩码选择性填充
# - 常用于注意力机制
# - 可以填充任意值

### 3.2 masked_select

# 条件选择
mask = tensor > 0
selected = torch.masked_select(tensor, mask)

# 特点：
# - 提取满足条件的元素
# - 返回一维张量
# - 用于数据过滤



## 4. 实际应用场景


In [122]:

### 4.1 自然语言处理

# 批量处理变长文本
texts = ["你好", "你好世界", "你好pytorch"]
# 转换为索引序列
sequences = [[1, 2], [1, 2, 3, 4], [1, 2, 5, 6, 7]]
# 填充处理
padded = torch.nn.utils.rnn.pad_sequence([torch.tensor(s) for s in sequences], 
                                       batch_first=True)


### 4.2 计算机视觉

# 图像批处理填充
images = [torch.randn(3, 224, 224), torch.randn(3, 256, 256)]
# 填充到相同大小
max_w, max_h = 0, 0
padded_images = [F.pad(img, 
                      (0, max_w - img.size(-1), 0, max_h - img.size(-2))) 
                 for img in images]


### 4.3 注意力机制

# 注意力掩码
scores = torch.randn(3, 4)
batch_size = 3
seq_len = 4
attention_mask = torch.ones(batch_size, seq_len).bool()
print(scores, scores.shape)
print(attention_mask, attention_mask.shape)

attention_mask = ~attention_mask.unsqueeze(1)
print(attention_mask, attention_mask.shape)

scores = scores.masked_fill(attention_mask, -float('inf'))
print(scores, scores.shape)



tensor([[-1.4625,  0.6699,  0.8244, -0.5473],
        [-0.0600, -0.6364,  0.6168,  0.5412],
        [-0.4060,  0.6554,  0.1888,  0.2390]]) torch.Size([3, 4])
tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]]) torch.Size([3, 4])
tensor([[[False, False, False, False]],

        [[False, False, False, False]],

        [[False, False, False, False]]]) torch.Size([3, 1, 4])
tensor([[[-1.4625,  0.6699,  0.8244, -0.5473],
         [-0.0600, -0.6364,  0.6168,  0.5412],
         [-0.4060,  0.6554,  0.1888,  0.2390]],

        [[-1.4625,  0.6699,  0.8244, -0.5473],
         [-0.0600, -0.6364,  0.6168,  0.5412],
         [-0.4060,  0.6554,  0.1888,  0.2390]],

        [[-1.4625,  0.6699,  0.8244, -0.5473],
         [-0.0600, -0.6364,  0.6168,  0.5412],
         [-0.4060,  0.6554,  0.1888,  0.2390]]]) torch.Size([3, 3, 4])


# 基本矩阵乘法

## 0. *符号或torch.mul 逐元素乘法运算

In [33]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])
c = a * b  # 结果: [4, 10, 18] 
print(c)
# 用途：逐元素乘法
# 特点：
# 逐元素操作
# 支持广播

tensor([ 4, 10, 18])


In [46]:
# 支持广播机制
A = torch.randn(2, 3, 4)
b = torch.randn(4)  # 将被广播到 [2, 3, 4]
C = A * b  # 可以运算
print(C, C.shape)
# 但要注意维度匹配
A1 = torch.randn(3, 4)
# 方法1: stack沿着新维度堆叠
A = torch.stack([A1, A1], dim=0)  # shape: [2, 3, 4]
# 或者用cat/vstack等其他方法:
# A = torch.cat([A1.unsqueeze(0), A1.unsqueeze(0)], dim=0)
B = torch.randn(1, 3, 4)  # 中间维度不匹配且不能广播
C = A * B  # 这会报错！
C

tensor([[[-0.1207, -0.4596, -1.7579,  0.6972],
         [ 0.4776,  0.5133,  5.5293,  0.1820],
         [ 0.6311,  0.2602,  0.2450, -0.2941]],

        [[-0.0852,  0.0782,  0.6269,  0.9053],
         [-0.4101, -0.2976,  4.9137, -0.5574],
         [ 0.5847, -0.3367,  2.1574,  1.6245]]]) torch.Size([2, 3, 4])


tensor([[[ 1.0555, -0.4722, -0.4096, -1.6197],
         [ 0.4276,  0.4707, -0.0342, -3.0549],
         [-0.0234,  1.5303,  1.2431,  0.0591]],

        [[ 1.0555, -0.4722, -0.4096, -1.6197],
         [ 0.4276,  0.4707, -0.0342, -3.0549],
         [-0.0234,  1.5303,  1.2431,  0.0591]]])

### 广播机制的基本原则
广播（Broadcasting）是PyTorch自动处理不同形状张量运算的一种机制，遵循以下规则：

从右向左对齐维度

维度为1的可以广播到任意大小

缺失的维度会自动补充为1

In [ ]:
# 示例1：标量和张量
a = torch.tensor([1, 2, 3])           # shape: [3]
b = 2                                 # shape: []
c = a * b                            # shape: [3]
print(c)
# b被广播到[3]：[2, 2, 2]

# 示例2：不同维度张量
x = torch.tensor([[1], [2], [3]])    # shape: [3, 1]
y = torch.tensor([4, 5, 6])          # shape: [3]
z = x + y                            # shape: [3, 3]
# x被广播到[3, 3]：[[1, 1, 1], [2, 2, 2], [3, 3, 3]]
print(z)

# 矩阵运算中的广播
batch1 = torch.randn(1, 3, 4)        # shape: [1, 3, 4]
batch2 = torch.randn(2, 3, 4)        # shape: [2, 3, 4]
result = batch1 + batch2             # shape: [2, 3, 4]
# batch1的第一维从1广播到2
# batch1被广播到[2, 3, 4]的shape
print(result)

tensor([2, 4, 6])
tensor([[5, 6, 7],
        [6, 7, 8],
        [7, 8, 9]])
tensor([[[-2.7964, -1.6647, -0.8599,  3.3309],
         [ 1.4293,  1.3316,  0.4652,  2.9003],
         [ 1.0565, -0.0409,  2.0614,  3.5634]],

        [[-2.6389, -2.0402,  1.6642,  0.9286],
         [ 0.4553, -0.0277,  1.8422,  1.9993],
         [-0.6844, -0.1489,  1.6114,  2.5484]]])


## 1. torch.matmul 或 @ 通用矩阵乘法运算符

In [23]:
A = torch.randn(2, 3, 4)
B = torch.randn(1, 4, 5)
C = torch.matmul(A, B)  # 或 C = A @ B
C.shape, C
# 用途：通用矩阵乘法，可以处理多种情况
# 特点：
# 支持广播机制（broadcasting）
# 可以处理不同维度的张量
# 根据输入维度自动选择合适的乘法方式
# 适用场景：
# 当不确定具体需要哪种矩阵乘法时
# 需要利用广播机制时

(torch.Size([2, 3, 5]),
 tensor([[[ 1.3635, -0.9821,  0.4979,  0.0790, -0.8105],
          [-0.4579,  1.3398, -0.2127,  0.2078,  0.3870],
          [ 1.0137,  2.6641,  3.4173, -0.7610,  1.2600]],
 
         [[-1.4139,  1.5463, -4.4714,  1.8290, -0.0223],
          [ 2.9376,  3.0884,  0.7397,  0.9518,  1.0776],
          [ 2.6754,  0.1563,  1.9222,  0.1724, -1.1409]]]))

## 2. torch.mm 标准2d矩阵的乘法运算

In [18]:
A = torch.randn(2, 3)
B = torch.randn(3, 4)
C = torch.mm(A, B)
C.shape, C
# 用途：二维矩阵乘法
# 特点：
# 只能处理2D张量
# 比matmul运行更快
# 不支持广播

(torch.Size([2, 4]),
 tensor([[ 1.1599,  0.9311, -0.0379, -0.5269],
         [-2.0701,  0.1375, -0.1265,  0.2984]]))

## 3. torch.bmm 专用于3d的矩阵乘法

In [20]:
A = torch.randn(2, 3, 4)
B = torch.randn(2, 4, 5)
C = torch.bmm(A, B)
C.shape, C
# 用途：批量矩阵乘法
# 特点：
# 只能处理3D张量
# 比matmul运行更快
# 不支持广播

(torch.Size([2, 3, 5]),
 tensor([[[-3.3989, -2.1715, -0.4937, -6.3157,  2.3156],
          [ 1.5177,  0.5540,  1.0551,  1.4014,  0.5849],
          [-0.1485, -0.0367, -0.2739, -0.9223, -0.3220]],
 
         [[-0.6912, -1.5945,  0.8291,  3.4820,  4.4763],
          [ 1.2197,  4.0576,  0.2921, -3.8629, -0.6839],
          [ 0.9048,  0.4832,  1.7965, -0.4360, -6.3062]]]))

## 4. torch.mv 适用于 2d与1d的相乘

In [19]:
A = torch.randn(2, 3)
B = torch.randn(3)
C = torch.mv(A, B)
C.shape, C
# 用途：矩阵乘以向量
# 特点：
# 只能处理2D矩阵和1D向量
# 适用场景：
# 线性变换
# 神经网络中的全连接层

(torch.Size([2]), tensor([-1.1414, -1.0777]))

# 张量堆叠和连接方法

1. **内存效率**：
   - `cat`和`stack`会创建新的内存
   - `view`和`expand`通常更内存效率

2. **速度**：
   - `vstack`/`hstack`比直接使用`cat`稍慢
   - `expand`比`repeat`更快但功能有限

3. **建议**：
   - 对于大量小张量，优先使用`stack`
   - 需要节省内存时，考虑使用`expand`
   - 处理动态形状时，`cat`更灵活


## 0.维度操作(view reshape)

主要区别：
1. 内存连续性：
   - `view`要求张量必须是连续的
   - `reshape`不要求连续，更灵活
   - `contiguous`用于确保连续性

2. 内存共享：
   - `view`总是共享内存
   - `reshape`尽可能共享内存，但不保证
   - `contiguous`在需要时会创建新的内存

3. 使用建议：
   - 如果确定张量是连续的，用`view`更高效
   - 如果不确定是否连续，用`reshape`更安全
   - 如果需要确保连续性，用`contiguous`


In [127]:
# 1. `view`操作：
# - 作用：改变张量的形状而不改变其数据
# - 特点：
#   - 要求张量在内存中是连续的（contiguous）
#   - 与原始张量共享内存
#   - 如果原始张量不是连续的，`view`会失败
# - 示例：

x = torch.randn(4, 4)
y = x.view(16)  # 将4x4变成16
print(y, y.shape)
y = x.view(-1, 8)  # -1表示自动计算该维度大小
print(y, y.shape)


# 2. `reshape`操作：
# - 作用：改变张量的形状
# - 特点：
#   - 更加灵活，不要求张量在内存中连续
#   - 如果可能，会尝试返回一个共享内存的视图
#   - 如果不可能共享内存，会返回一个新的张量
# - 示例：

x = torch.randn(4, 4)
print(x, x.shape)
y = x.reshape(2, 8)  # 总是能工作，即使x不是连续的
print(y, y.shape)

# 3. `contiguous`操作：
# - 作用：确保张量在内存中是连续的
# - 特点：
#   - 如果张量已经是连续的，返回自身
#   - 如果不是连续的，返回一个新的连续张量
# - 示例：

x = torch.randn(4, 4)
x = x.transpose(0, 1)  # transpose后可能不连续
x = x.contiguous()  # 确保连续



tensor([ 0.6305,  0.3605, -0.0297, -0.6499, -0.1706, -0.2751,  0.3029,  0.4524,
        -0.9376, -0.0430,  0.4875, -1.4365, -0.0883,  0.7324,  0.8828, -2.3603]) torch.Size([16])
tensor([[ 0.6305,  0.3605, -0.0297, -0.6499, -0.1706, -0.2751,  0.3029,  0.4524],
        [-0.9376, -0.0430,  0.4875, -1.4365, -0.0883,  0.7324,  0.8828, -2.3603]]) torch.Size([2, 8])
tensor([[-0.1914, -0.8185,  0.8679,  2.1392],
        [-0.9743, -0.5182, -0.5222,  0.9768],
        [-1.7901,  1.8443,  0.3530, -1.6218],
        [ 0.0425, -0.2054, -2.1093, -1.8665]]) torch.Size([4, 4])
tensor([[-0.1914, -0.8185,  0.8679,  2.1392, -0.9743, -0.5182, -0.5222,  0.9768],
        [-1.7901,  1.8443,  0.3530, -1.6218,  0.0425, -0.2054, -2.1093, -1.8665]]) torch.Size([2, 8])


In [ ]:

# 多头注意力中的维度变换
x = torch.randn(batch_size, seq_len, hidden_dim)
# 方法1：使用view（如果x是连续的）
x = x.view(batch_size, seq_len, num_heads, head_dim)

# 方法2：使用reshape（更安全）
x = x.reshape(batch_size, seq_len, num_heads, head_dim)

# 方法3：确保连续后使用view 
x = x.contiguous().view(batch_size, seq_len, num_heads, head_dim)

In [ ]:
### unsqueeze (增加维度)

x = torch.randn(3, 4)
x_new = x.unsqueeze(0)  # shape: [1, 3, 4]
print(x_new.shape)
# 或
x_new = x.unsqueeze(-1)  # shape: [3, 4, 1]
print(x_new.shape)


torch.Size([1, 3, 4])
torch.Size([3, 4, 1])


In [ ]:
### squeeze (移除维度) squeeze()在默认情况下会删除所有维度大小为1的维度。让我详细解释一下：
x = torch.randn(1, 3, 1, 4, 1)
x_new = x.squeeze(0)    # 只删除第0维
print(x_new.shape)      # torch.Size([3, 1, 4, 1])

x_new = x.squeeze(2)    # 只删除第2维
print(x_new.shape)      # torch.Size([1, 3, 4, 1])

# 如果指定维度的大小不为1，则不会发生任何变化
x_new = x.squeeze(1)    # 第1维大小为3，不会被删除
print(x_new.shape)      # torch.Size([1, 3, 1, 4, 1])

x.squeeze_()  # 直接修改x，不返回新张量 实际上，似乎所有后面带上_的函数都是直接修改张量，不返回新张量



torch.Size([3, 4])
torch.Size([3, 1, 4])


## 1. 堆叠操作 (Stacking)

In [54]:
### torch.stack
# 在新维度上堆叠
tensors = [torch.randn(3, 4) for _ in range(5)]
stacked = torch.stack(tensors, dim=0)  # shape: [5, 3, 4]
print(stacked.shape)
stacked = torch.stack(tensors, dim=1)  # shape: [3, 5, 4]
print(stacked.shape)

torch.Size([5, 3, 4])
torch.Size([3, 5, 4])


## 2. 连接操作 (Concatenation)


In [58]:
### torch.cat

# 在现有维度上连接
a = torch.randn(3, 4)
b = torch.randn(3, 4)
c = torch.cat([a, b], dim=0)  # shape: [6, 4]
c = torch.cat([a, b], dim=1)  # shape: [3, 8]
c.shape


torch.Size([3, 8])

## 3. *stack操作

In [ ]:

### torch.vstack (垂直堆叠)

# 等价于 torch.cat([a, b], dim=0)
a = torch.randn(3, 4)
b = torch.randn(3, 4)
c = torch.vstack([a, b])  # shape: [6, 4]


### torch.hstack (水平堆叠)

# 等价于 torch.cat([a, b], dim=1)
a = torch.randn(3, 4)
b = torch.randn(3, 4)
c = torch.hstack([a, b])  # shape: [3, 8]


### torch.dstack (深度堆叠)

# 在深度方向堆叠
a = torch.randn(3, 4)
b = torch.randn(3, 4)
c = torch.dstack([a, b])  # shape: [3, 4, 2]

## 3. 复制和重复

In [66]:
### repeat

x = torch.tensor([[1, 2, 3]])  # shape: [1, 3]
# 在各个维度上重复
y = x.repeat(2, 3)  # shape: [4, 9]
y[0, 0] = 9  # 修改y不会影响x，因为是独立的副本
print(x)  # 仍然是[[1, 2, 3]]
print(y)

### expand

x = torch.tensor([[1, 2, 3]])
# 广播式扩展，不复制数据
y = x.expand(4, 3)  # shape: [4, 3]
y[0, 0] = 9  # 修改y会影响x，因为共享内存
print(x)  # 也会变成
print(y)


tensor([[1, 2, 3]])
tensor([[9, 2, 3, 1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3, 1, 2, 3]])
tensor([[9, 2, 3]])
tensor([[9, 2, 3],
        [9, 2, 3],
        [9, 2, 3],
        [9, 2, 3]])


## 5. 实战中的常用组合


### 1. 批处理数据准备


In [70]:
# 单样本转批量
single_sample = torch.randn(3, 4)
batch = single_sample.unsqueeze(0)  # 添加批次维度 [1, 3, 4]

# 多个样本组批
samples = [torch.randn(3, 4) for _ in range(5)]
batch = torch.stack(samples, dim=0)  # [5, 3, 4]



### 2. 特征拼接


In [72]:
# 在特征维度上拼接
feature1 = torch.randn(32, 10)
feature2 = torch.randn(32, 20)
combined = torch.cat([feature1, feature2], dim=1)  # [32, 30]
combined_hstack = torch.hstack([feature1, feature2])  # [32, 30]
combined.shape, combined_hstack.shape

(torch.Size([32, 30]), torch.Size([32, 30]))


### 3. 序列处理


In [ ]:
# 处理变长序列
sequences = [torch.randn(length, 128) for length in [5, 3, 4]]
padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)


### 4. 图像处理


In [ ]:

# 添加通道维度
image = torch.randn(28, 28)
with_channel = image.unsqueeze(0)  # [1, 28, 28]

# 批量处理
images = [torch.randn(1, 28, 28) for _ in range(4)]
batch = torch.cat(images, dim=0)  # [4, 1, 28, 28]



In [69]:

### 5. 注意力机制

# 多头注意力的头拆分
batch_size, n_heads = 32, 8
hidden_dim = 512
x = torch.randn(batch_size, hidden_dim)
print(x.shape)
# 重塑为多头形式
reshaped = x.view(batch_size, n_heads, hidden_dim // n_heads)
print(reshaped.shape)


torch.Size([32, 512])
torch.Size([32, 8, 64])
